# SSSP Example

In [ ]:
import dask
import numpy as np
import grblas as gb
import dask_grblas as dgb
from grblas import op

In [ ]:
# Create random data
N = 1000
num_chunks = 4
r = np.random.rand(N, N) < 0.001
r = r | r.T  # symmetric
r = r & ~np.diag(np.ones(N, dtype=bool))  # no self edges

In [ ]:
chunks = np.array_split(r, num_chunks, axis=0)

In [ ]:
def to_matrix(chunk):
    rows, cols = np.nonzero(chunk)
    values = np.random.rand(rows.size)
    return dgb.Matrix.from_values(
        rows,
        cols,
        values,
        nrows=chunk.shape[0],
        ncols=chunk.shape[1]
    )
delayed_chunks = [to_matrix(chunk) for chunk in chunks]

In [ ]:
A = dgb.row_stack(delayed_chunks)
sources = dgb.Vector.from_values(
    np.random.randint(N),
    0,
    size=N,
    dtype=A.dtype
)

In [ ]:
# Calculate expected with grblas
B = A.compute()
v = sources.dup().compute()
v_dup = gb.Vector.new(v.dtype, size=N)
i = 0
while True:
    i += 1
    v_dup << v
    v(op.min) << B.mxv(v, op.min_plus)
    if v.isequal(v_dup):
        break
expected = v
i

In [ ]:
# Calculate with dask-grblas
i = 0
v = sources.dup()
while True:
    i += 1
    v_dup = v.dup()
    v(op.min) << A.mxv(v, op.min_plus)
    # persist so we don't recompute every iteration (threaded scheduler)
    v._delayed = v._delayed.persist()
    if v.isequal(v_dup):
        break
i

In [ ]:
assert expected.isequal(v.compute())

In [ ]:
expected